In [2]:
import fnmatch
import pandas as pd

In [14]:
gatewayVolumesLoc = "./GatewayVolumes_2056.csv"
chainFlowVolumesLoc = "./INITFMCC_ChainFlowShare_Y2056_WFY_Funded_Op2_001.CSV"
postcodeSharesLoc = "./PostcodeShares_2056.csv"

In [15]:
gatewayVolumes = pd.read_csv(gatewayVolumesLoc)
chainFlowVolumes = pd.read_csv(chainFlowVolumesLoc)
postcodeShares = pd.read_csv(postcodeSharesLoc)
postcodeShares['Postcode'] = postcodeShares['Postcode'].astype(str)
gatewayMatchColNames = ['Gateway','Sector','Import / Export','Full / Empty']

In [16]:
yearColName = gatewayVolumes.columns[pd.Series(gatewayVolumes.columns).str.startswith('Y2')][0]
if not yearColName in postcodeShares.columns:
    print('INPUT DATA ERROR: Column ' + yearColName + ' is not present in all the necessary input files.\n')

# this function separates strings by semicolon and matches with wildcard
def matchStrings(primaryString, secondaryString):
    isMatch = False
    secondaryStringList = secondaryString.split(';')
    for i in range(len(secondaryStringList)):
        isMatch = isMatch | fnmatch.fnmatch(primaryString, secondaryStringList[i])
    return isMatch

# this function carries out matchStrings for all required columns to get a full match
def matchCols(primaryRow, secondaryRow, primaryColsList, secondaryColsList):
    rowMatch = True
    for j in range(len(primaryColsList)):
        rowMatch = rowMatch & matchStrings(primaryRow[primaryColsList[j]], secondaryRow[secondaryColsList[j]])
    return rowMatch


In [24]:
from IPython.display import display  # For pretty printing in Jupyter

allPostcodeVolumes = pd.DataFrame()

def pretty_print_df(df, title="DataFrame", max_rows=10):
    """Display a DataFrame with a title and limited rows"""
    print(f"\n{'='*50}\n{title}\n{'='*50}")
    display(df.head(max_rows))
    print(f"Shape: {df.shape}")
    if len(df) > max_rows:
        print(f"Showing first {max_rows} rows of {len(df)} total rows")

for gatewayIt in range(len(gatewayVolumes)):
    gatewayRow = gatewayVolumes.iloc[gatewayIt]
    print(f"\nCurrent Gateway: {gatewayRow['Gateway']} (Values: {list(gatewayRow.values)})")
    
    # Match postcode shares
    matchedShares = postcodeShares[postcodeShares\
                                   .apply(lambda row:\
                                        matchCols(gatewayRow,\
                                        row, gatewayMatchColNames,\
                                        gatewayMatchColNames),\
                                        axis=1)
                                  ]\
                                .drop(gatewayMatchColNames, axis=1)
    # Match chain flow volumes
    matchedFlows = chainFlowVolumes[chainFlowVolumes\
                                    .apply(lambda row:\
                                         matchCols(gatewayRow,\
                                         row, gatewayMatchColNames,\
                                         gatewayMatchColNames), axis=1)
                                   ]
    if len(matchedShares) == 0:
        print(f'INPUT DATA ERROR: No postcode shares corresponding to gateway {gatewayRow["Gateway"]}\n')
        continue

    if len(matchedFlows) == 0:
        print(f'INPUT DATA ERROR: No chain flow volumes corresponding to gateway {gatewayRow["Gateway"]}\n')
        continue
    
    # Display initial matches
    print(f"\nMatched Shares for Gateway: {gatewayRow['Gateway']}")
    pretty_print_df(matchedShares,"Matched Postcode Shares" )

    print(f"Matched Flows for Gateway: {gatewayRow['Gateway']}")
    pretty_print_df(matchedFlows,"Matched Chain Flow Volumes")

    # Check for state mismatches
    mismatched = matchedFlows.filter(['Volume Entry', 'From', 'To', 'From State', 'To State'])
    mismatched = mismatched.merge(mismatched.filter(['To', 'To State']),\
                                  left_on=['From', 'From State'],\
                                  right_on=['To', 'To State'], how='left')
    
    mismatched = mismatched[mismatched['From'] != mismatched['Volume Entry']]
    mismatched['To State_y'] = mismatched['To State_y'].fillna('')
    mismatched = mismatched[mismatched['To State_y'] == '']
    
    if len(mismatched) > 0:
        print(f'INPUT DATA ERROR: State mismatch from {mismatched["From"].iloc[0]} to {mismatched["To_x"].iloc[0]}. From State and To State must match.')
        display(mismatched.style.set_caption("Mismatched States"))
        break

    # Merge shares with flows
    print('\nMerging shares with chain flows...')
    shareVolumes = matchedShares.merge(matchedFlows,\
                                       left_on='Chain Link',\
                                       right_on='To', how='right')\
                                .rename(columns={'Freight Area': 'To Freight Area',\
                                                 'Postcode': 'To Postcode',\
                                                 yearColName: 'To Share'})\
                                .filter(gatewayMatchColNames +\
                                        ['Industry Class', 'To Freight Area',\
                                         'To Postcode', 'To Share', 'From', 'To',\
                                         'From State', 'To State', 'Run Type', 'Output Link Share'])
    
    print("Merged Share Volumes (Initial):")
    pretty_print_df(shareVolumes,"Initial Share Volumes")
    
    # Calculate input combined shares
    print('\nGetting input combined shares...')
    shareVolumes['Input Combined Shares'] =\
                shareVolumes['Output Link Share'] * shareVolumes['To Share']

    currentScale = shareVolumes\
                    .groupby(['To', 'To State'])\
                    .agg({'Input Combined Shares': 'sum'})\
                    .reset_index()\
                    .rename(columns={'Input Combined Shares': 'Current Scale'})
    
    goalScale = matchedFlows\
                .groupby(['To', 'To State'])\
                .agg({'Output Link Share': 'sum'})\
                .reset_index()\
                .rename(columns={'Output Link Share': 'Goal Scale'})
    
    shareVolumes = shareVolumes.merge(currentScale).merge(goalScale)
    shareVolumes['Input Combined Shares'] =\
                shareVolumes['Input Combined Shares'] / \
                shareVolumes['Current Scale'] * shareVolumes['Goal Scale']

    # Display scaling info
    print("Scaling Information:")
    pretty_print_df(currentScale,"Current Scale")

    print("Goal Scale:")
    pretty_print_df(goalScale,"Goal Scale")

    shareVolumes.loc[(shareVolumes['To State'].isin(shareVolumes['From State'])),\
                     'Output Combined Shares'] =\
                    shareVolumes['Input Combined Shares']
    
    shareVolumes = shareVolumes.filter(gatewayMatchColNames +\
                   ['Industry Class', 'To', 'To Freight Area', 'To State',\
                    'To Postcode', 'Input Combined Shares', 'Output Combined Shares'])

    print("\nShare Volumes after Scaling:")
    pretty_print_df(shareVolumes,"Scaled Share Volumes")
   
    #     Add gateway to main table
    print('\nAdding gateway to main table for output combined shares...')
    entryPoint = matchedFlows['Volume Entry'].iloc[0]
    gatewayRows = matchedShares[matchedShares['Chain Link'] == entryPoint]\
                .rename(columns={'Chain Link': 'To', 'Postcode': 'To Postcode',\
                                 'Freight Area': 'To Freight Area',\
                                 yearColName: 'Output Combined Shares'})

    for i in range(len(gatewayMatchColNames)):
        gatewayRows[gatewayMatchColNames[i]] = gatewayRow[gatewayMatchColNames[i]]
    
    gatewayRows['Industry Class'] = gatewayRow['Industry Class']
    gatewayRows['To State'] = matchedFlows[matchedFlows['From'] == entryPoint].iloc[0].loc['From State']
    
    shareVolumes = pd.concat([shareVolumes, gatewayRows], ignore_index=True)

    shareVolumes = shareVolumes.groupby(gatewayMatchColNames +\
                                        ['Industry Class', 'To', 'To Postcode', 'To State'])\
                               .agg({'Input Combined Shares': 'sum', 'Output Combined Shares': 'sum'})\
                               .reset_index()

    print("Share Volumes with Gateway Entry:")
    pretty_print_df(shareVolumes,"Grouping by node...")
  
    
    shareVolumes['Input Volume'] = shareVolumes['Input Combined Shares'] * gatewayRow[yearColName]
    shareVolumes['Output Volume'] = shareVolumes['Output Combined Shares'] * gatewayRow[yearColName]

    shareVolumes = shareVolumes[gatewayMatchColNames +\
                                ['Industry Class', 'To', 'To Postcode',\
                                 'To State', 'Input Volume', 'Output Volume']]

    print("Final Share Volumes with Absolute Volumes:")
    pretty_print_df(shareVolumes,f"Final Volumes for {gatewayRow['Gateway']}")
    print(shareVolumes["Input Volume"].sum())
    print(shareVolumes["Output Volume"].sum())
    
    allPostcodeVolumes = pd.concat([allPostcodeVolumes, shareVolumes], ignore_index=True)
    break

# Final cumulative result
print("\nAll Postcode Volumes (Cumulative Result):")
display(allPostcodeVolumes.style.set_caption("All Gateway Postcode Volumes")\
        .format({'Input Combined Shares': '{:.4f}', 'Output Combined Shares': '{:.4f}',\
                 'Input Volume': '{:.2f}', 'Output Volume': '{:.2f}'}))



Current Gateway: Swanson Dock West (Values: ['Swanson Dock West', 'I&MC', 'Import', 'Full', 'IMEX', 4209.470819953])

Matched Shares for Gateway: Swanson Dock West

Matched Postcode Shares


,Chain Link,Freight Area,Postcode,Y2056
0,Swanson Dock West,Port of Melbourne - Swanson Dock West,3003,1.0
1,Swanson Dock East,Port of Melbourne - Swanson Dock East,3003,1.0
2,Webb Dock North,Port of Melbourne - Webb Dock North,3207,1.0
3,Webb Dock South,Port of Melbourne - Webb Dock South,3207,1.0
4,Appleton Dock,Port of Melbourne - Appleton Dock,3003,1.0
5,Dynon North Rail Terminal,Dynon North Rail Terminal,3003,1.0
6,Dynon South Rail Terminal,Dynon South Rail Terminal,3003,1.0
7,Altona Rail Terminal,Altona Rail Terminal,3018,1.0
8,WIFT West Rail Terminal,WIFT West Rail Terminal,3029,1.0
9,WIFT East Rail Terminal,WIFT East Rail Terminal,3029,1.0


Shape: (80, 4)
Showing first 10 rows of 80 total rows
Matched Flows for Gateway: Swanson Dock West

Matched Chain Flow Volumes


,Gateway,Sector,Import / Export,Full / Empty,Industry Class,Volume Entry,From,To,From State,To State,Run Type,Output Link Share
0,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,PRS Terminals WIFT / BIFT,Full,Full,RL,0.121442
1,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,PRS Terminals Others,Full,Full,RL,0.083671
2,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,Importers,Full,Empty,CR,0.176977
3,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Swanson Dock West,Transport Depots,Full,Full,BR,0.617909
4,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Empty,CR,0.085009
5,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals WIFT / BIFT,Importers,Full,Complete,IG,0.036433
6,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals Others,Importers,Full,Empty,CR,0.075304
7,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,PRS Terminals Others,Importers,Full,Complete,IG,0.008367
8,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Transport Depots,Importers,Full,Empty,CR,0.617909
9,Swanson Dock West,I&MC,Import,Full,IMEX,Swanson Dock West,Importers,PRS Terminals WIFT / BIFT,Empty,Empty,CR,0.034800


Shape: (17, 12)
Showing first 10 rows of 17 total rows

Merging shares with chain flows...
Merged Share Volumes (Initial):

Initial Share Volumes


...

Shape: (10, 3)
Goal Scale:

Goal Scale


,To,To State,Goal Scale
0,Container Parks,Complete,0.649848
1,Importers,Complete,0.044800
2,Importers,Empty,0.955200
3,PRS Terminals Others,Empty,0.023977
4,PRS Terminals Others,Full,0.083671
5,PRS Terminals WIFT / BIFT,Empty,0.034800
6,PRS Terminals WIFT / BIFT,Full,0.121442
7,Swanson Dock West,Complete,0.305352
8,Transport Depots,Empty,0.493150
9,Transport Depots,Full,0.617909


Shape: (10, 3)

Share Volumes after Scaling:

Scaled Share Volumes


,Gateway,Sector,Import / Export,Full / Empty,Industry Class,To,To Freight Area,To State,To Postcode,Input Combined Shares,Output Combined Shares
0,Swanson Dock West,I&MC,Import,Full,IMEX,PRS Terminals WIFT / BIFT,WIFT West Rail Terminal,Full,3029,0.065283,0.065283
1,Swanson Dock West,I&MC,Import,Full,IMEX,PRS Terminals WIFT / BIFT,WIFT East Rail Terminal,Full,3029,0.033633,0.033633
2,Swanson Dock West,I&MC,Import,Full,IMEX,PRS Terminals WIFT / BIFT,BIFT North Rail Terminal,Full,3753,0.014076,0.014076
3,Swanson Dock West,I&MC,Import,Full,IMEX,PRS Terminals WIFT / BIFT,BIFT South Rail Terminal,Full,3753,0.008451,0.008451
4,Swanson Dock West,I&MC,Import,Full,IMEX,PRS Terminals Others,Altona Rail Terminal,Full,3018,0.012169,0.012169
5,Swanson Dock West,I&MC,Import,Full,IMEX,PRS Terminals Others,Dandenong South Rail Terminal,Full,3975,0.048976,0.048976
6,Swanson Dock West,I&MC,Import,Full,IMEX,PRS Terminals Others,Somerton Rail Terminal,Full,3062,0.022526,0.022526
7,Swanson Dock West,I&MC,Import,Full,IMEX,Importers,NaN,Empty,3003,0.009372,0.009372
8,Swanson Dock West,I&MC,Import,Full,IMEX,Importers,NaN,Empty,3011,0.012878,0.012878
9,Swanson Dock West,I&MC,Import,Full,IMEX,Importers,NaN,Empty,3012,0.031165,0.031165


Shape: (327, 11)
Showing first 10 rows of 327 total rows

Adding gateway to main table for output combined shares...
               Gateway Sector Import / Export Full / Empty Industry Class  \
0    Swanson Dock West   I&MC          Import         Full           IMEX   
1    Swanson Dock West   I&MC          Import         Full           IMEX   
2    Swanson Dock West   I&MC          Import         Full           IMEX   
3    Swanson Dock West   I&MC          Import         Full           IMEX   
4    Swanson Dock West   I&MC          Import         Full           IMEX   
..                 ...    ...             ...          ...            ...   
122  Swanson Dock West   I&MC          Import         Full           IMEX   
123  Swanson Dock West   I&MC          Import         Full           IMEX   
124  Swanson Dock West   I&MC          Import         Full           IMEX   
125  Swanson Dock West   I&MC          Import         Full           IMEX   
126  Swanson Dock West   I&MC       

In [20]:
allPostcodeVolumes

,Gateway,Sector,Import / Export,Full / Empty,Industry Class,To,To Postcode,To State,Input Volume,Output Volume
0,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3003,Complete,239.223621,0.000000
1,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3011,Complete,304.105831,0.000000
2,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3012,Complete,1031.777174,0.000000
3,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3013,Complete,32.208107,0.000000
4,Swanson Dock West,I&MC,Import,Full,IMEX,Container Parks,3018,Complete,187.390876,0.000000
...,...,...,...,...,...,...,...,...,...,...
1674,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3061,Full,8.384297,8.384297
1675,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3062,Empty,0.000000,37.130458
1676,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3062,Full,5.030578,5.030578
1677,BIFT South Rail Terminal,ICDR,Outbound,Full,ICDR,Transport Depots,3753,Empty,0.000000,148.521832


In [9]:
allPostcodeVolumes

""
